The main workspace for calculating pseudoinverse $A^\dagger$ of nonsymmetric matrices

In [15]:
workspace()

In [24]:
using Plots
using MatrixDepot   # for downloading and loading matrices from the UF sparse matrix collection
using StatsBase
using JLD  
include("../src/Pseudoinverse.jl")
importall Pseudoinverse

In [18]:
# Uniform random
m = 50;    # number of columns
n= 200;    # number of rows
r = 20;    # rankd
prob = uniform_mat_rank(m,n,r);

In [26]:
# Rank deficient matrices from UF sparse matrix collection
# NYPA/Maragal_3, NYPA/Maragal_4, NYPA/Maragal_5, 
# Meszaros/primagaz  LPnetlib/lp_fit2d  LPnetlib/lp_ken_07
name = "NYPA/Maragal_3"  
#matrixdepot(name, :get);    # Use this for first time call
A = matrixdepot(name, :read);
exacterror =false;
sA= size(A); m= sA[1] ; n = sA[2];
prob = Prob(A,[],[], [], name);
size(A)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 34  120k   34 42944    0     0  46937      0  0:00:02 --:--:--  0:00:02 54497

download:/home/robert/.julia/v0.5/MatrixDepot/src/../data/uf/NYPA/Maragal_3.tar.gz


100  120k  100  120k    0     0   107k      0  0:00:01  0:00:01 --:--:--  120k


Maragal_3/Maragal_3.mtx
Maragal_3/Maragal_3_b.mtx


(1690,860)

In [27]:
#Basic parameters
tol = 10.0^(-2); restol = 10.0^(-5);
maxiter=1000;
max_time = 100;
printiters = true;
exacterror =false;

In [28]:
skip_error_calculation = 1;   # number of iterations where error is not calculated (to save time!) ceil((m)^(1/2))
options = MyOptions(n,m,0,"uni",tol,restol,maxiter,skip_error_calculation,max_time,printiters,exacterror, "none") 

Pseudoinverse.MyOptions(860,1690,0,"uni",0.01,1.0e-5,1000,1,100.0,true,false,"none")

In [ ]:
# Testing three methods
options.max_time = 20; options.restol = 10.0^(-8);
OUTPUTS = [];
method_name = "SATAX";#"SATAX"; "NewtonSchultz" "SAXAS"
options.sketch = "uni";
options.M0type = "ATproj";   # Select initial matrix, options: ATproj, eye
output= pseudoinvert_matrix(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

options.sketch = "ada";
options.M0type = "ATproj";
output= pseudoinvert_matrix(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];

method_name = "NewtonSchulz";
output= pseudoinvert_matrix(prob, method_name, options);
OUTPUTS = [OUTPUTS ; output];


In [22]:
savename = replace(prob.name, r"[\/]", "-");
save("../data/$(savename).jld", "OUTPUTS", OUTPUTS)

In [32]:
pgfplots()
plot_outputs_Plots(OUTPUTS,savename)

LoadError: LoadError: UndefVarError: pgfplots not defined
while loading In[32], in expression starting on line 1

In [3]:
# Loading saved problem data
probname = "rcv1_train.binary-ridge"#"gisette_scale-ridge";  
test = load("./data/$(probname).jld")
OUTPUTS = test["OUTPUTS"];
# mask = [1,3,5]
# OUTPUTS[mask]